In [1]:
import pandas as pd

In [36]:
import numpy as np

In [175]:
col_names = ["Class", "Age","Gender", "Alive", "Name", "Don't Know"]
titanic_df = pd.read_csv(r'C:\Users\Yogen\Documents\1. Data Science Courses\2. MIT Data Science Course\Lecture13\TitanicPassengers.txt', header=None, names = col_names)

In [176]:
titanic_df = titanic_df.drop(["Name", "Don't Know"], axis=1)

In [177]:
titanic_df = pd.get_dummies(titanic_df)

In [178]:
titanic_df.head()

,Class,Age,Alive,Gender_F,Gender_M
0,1,29.00,1,1,0
1,1,0.92,1,0,1
2,1,2.00,0,1,0
3,1,30.00,0,0,1
4,1,25.00,0,1,0


### Scaling age

In [48]:
def z_scaling(vals):
    vals = np.array(vals)
    mean, std = np.mean(vals), np.std(vals)
    z = (vals-mean)/std
    return z

### K nearest neighbours

In [58]:
from sklearn.metrics import confusion_matrix
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split

In [179]:
y = titanic_df["Alive"].values # y values isolated

In [180]:
y

array([1, 1, 0, ..., 0, 0, 0], dtype=int64)

In [181]:
x = titanic_df.drop(["Alive"], axis=1) #x values isolated

In [182]:
x.head()

,Class,Age,Gender_F,Gender_M
0,1,29.00,1,0
1,1,0.92,0,1
2,1,2.00,1,0
3,1,30.00,0,1
4,1,25.00,1,0


In [183]:
x = x.apply(z_scaling) #x values scaled

In [220]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2)

In [221]:
x_train

,Class,Age,Gender_F,Gender_M
935,0.942276,0.147076,-0.767897,0.767897
470,-0.246651,-0.130575,-0.767897,0.767897
48,-1.435579,-0.130575,-0.767897,0.767897
286,-0.246651,0.008251,-0.767897,0.767897
1043,0.942276,-0.234694,-0.767897,0.767897
...,...,...,...,...
279,-1.435579,1.465916,-0.767897,0.767897
853,0.942276,-0.477638,1.302258,-1.302258
89,-1.435579,1.257678,1.302258,-1.302258
78,-1.435579,-0.199987,-0.767897,0.767897


##### KNN

In [131]:
knn = KNeighborsClassifier(n_neighbors=5, metric='euclidean')

In [132]:
knn.fit(x_train, y_train)

KNeighborsClassifier(metric='euclidean')

In [133]:
y_pred = knn.predict(x_test)

In [116]:
pd.DataFrame(confusion_matrix(y_test, y_pred))

,0,1
0,128,25
1,28,81


### Best k

In [198]:
def senstivity_and_specificity(y_test, y_pred):
    confusion_matrix_df = pd.DataFrame(confusion_matrix(y_test, y_pred))
    specificity = confusion_matrix_df.iloc[0,0]/confusion_matrix_df.sum(axis=1)[0]
    
    senstivity = confusion_matrix_df.iloc[1,1]/confusion_matrix_df.sum(axis=1)[1]
    
    return (senstivity, specificity)

In [118]:
list(senstivity_and_specificity(y_test, y_pred))

[0.8366013071895425, 0.7431192660550459]

In [199]:
k = [3,5,7,9,11]
results = {}
for nearest_neighbours in k:
    knn = KNeighborsClassifier(n_neighbors=nearest_neighbours, metric = "euclidean")
    knn.fit(x_train, y_train)
    y_pred = knn.predict(x_test)
    results[nearest_neighbours] = senstivity_and_specificity(y_test, y_pred)

In [201]:
results #inconclusive, I guess

{3: (0.6822429906542056, 0.864516129032258),
 5: (0.616822429906542, 0.9096774193548387),
 7: (0.616822429906542, 0.8903225806451613),
 9: (0.6355140186915887, 0.8709677419354839),
 11: (0.616822429906542, 0.8709677419354839)}

### Logistic Regression

In [203]:
from sklearn.linear_model import LogisticRegression

In [222]:
lm = LogisticRegression()
lm.fit(x_train, y_train)

LogisticRegression()

In [223]:
y_pred_lm = lm.predict(x_test)

In [224]:
senstivity_and_specificity(y_test, y_pred_lm)

(0.6847826086956522, 0.8135593220338984)

In [212]:
lm.coef_

array([[-1.12087892, -0.58834367,  0.63956788, -0.63956788]])

##### Doing LR for multiple splits of 80-20

In [237]:
senstivity_list =[]
specificity_list = []
for i in range(50):
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2)
    lm = LogisticRegression()
    lm.fit(x_train, y_train)
    y_pred_lm = lm.predict(x_test)
    senstivity, specificity = senstivity_and_specificity(y_test, y_pred_lm)
    senstivity_list.append(senstivity)
    specificity_list.append(specificity)

In [238]:
np.mean(senstivity_list)

0.7041797834309843

In [239]:
np.mean(specificity_list)

0.8374216990313366